# Event sourcing

Instead of writing directly to the tables, the client writes to an extra table, the `eventstore`, to keep track of all the changes. This allows now only to keep track of the changes, but also allows to revert the database to what it was at any point of time.

Here is what it looks like:

In [1]:
import mdb

client = mdb.MDBClient(hostname='localhost',
                       username='postgres',
                       password='',
                       database='mdb')

es = client.get('eventstore')
es

/Users/tga/miniconda3/envs/mdb/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:3010: SAWarning: Did not recognize type 'xml' of column 'xdl'
  "Did not recognize type '%s' of column '%s'" % (attype, name)
/Users/tga/miniconda3/envs/mdb/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.conformer, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
/Users/tga/miniconda3/envs/mdb/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.molecule, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
/Users/tga/miniconda3/envs/mdb/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This

,type,id,uuid,data,event,timestamp
0,molecule,18,e3dde6c2-ab0c-4c77-9d55-e52d58a1425d,{'smiles': 'DEF'},create,2020-02-17 03:10:16.722259
1,xy_data,17,4d78fcc7-27d1-419d-8b0d-a882f52ac962,"{'x': [0.0, 0.10101010101010101, 0.20202020202...",create,2020-02-17 03:10:00.699414
2,data_unit,16,69cde87a-5fe2-428e-b129-4dad9b8bbe4f,{'name': 'stupid_units'},create,2020-02-17 03:10:00.647864
3,experiment,15,0e0e76e9-6f2f-40fd-84e4-cdcc05de0f8e,"{'notes': '', 'metadata': {'param1': 100, 'kno...",create,2020-02-17 03:09:59.145749
4,experiment_machine,14,455250de-954e-47e9-b541-55215d07d881,"{'name': 'Mad NMR', 'lab_id': 'cd599368-4fd8-4...",create,2020-02-17 03:09:59.105570
5,experiment_type,13,3cc2423a-7a6a-4aa7-8edc-c1b41d5e223d,{'name': 'NMR'},create,2020-02-17 03:09:58.637635
6,synthesis,12,bd8e2ba9-5f1a-40f8-8f9b-af4c47b70212,{'notes': ' # Synthesis of ABC Everything wen...,update,2020-02-17 03:09:57.003529
7,synthesis,11,bd8e2ba9-5f1a-40f8-8f9b-af4c47b70212,"{'hid': 'MAD_2020-02-17_0', 'xdl': '<xdl><reci...",create,2020-02-17 03:09:55.410886
8,molecule,10,c053009a-9b1d-44a7-8267-f5befdb03eb0,"{'smiles': 'ABC', 'metadata': {}}",update,2020-02-17 03:09:48.810674
9,molecule_fragment,7,cf8151dd-2e19-43a3-a175-4e3293f858d5,"{'order': 0, 'fragment_id': '93d1dd17-8f7b-4f6...",create,2020-02-17 03:09:48.748453


## Rollback

To revert at a specific point of time, the client implements a method `rollback`. 

Here we will revert the database to everything before the timestamp `2020-02-17 00:21:59.810407` (creation of data_unit).

In [2]:
client.rollback(before=es.at[3, 'timestamp'].to_pydatetime())

In [8]:
client.get('eventstore').sort_values(by=['id'])

,event,data,id,type,uuid,timestamp
49,create,"{'name': 'Mad Lab', 'short_name': 'MAD'}",1,lab,cd599368-4fd8-4db5-b0d2-d70b8af2a1b5,2020-02-17 03:09:48.219183
48,create,"{'name': 'Mad Machine Doing Chemistry', 'lab_i...",2,synthesis_machine,27811cc0-6c7b-4fb0-b679-435e1e280c1a,2020-02-17 03:09:48.400127
47,create,{'smiles': 'A'},3,fragment,93d1dd17-8f7b-4f61-8cb4-dbeab0e7730f,2020-02-17 03:09:48.509270
46,create,{'smiles': 'B'},4,fragment,e23166bb-88c4-464b-ac27-384e7ac7a475,2020-02-17 03:09:48.571496
45,create,{'smiles': 'C'},5,fragment,10e88031-39e6-4a60-a760-55ae2d142e43,2020-02-17 03:09:48.631793
44,create,{'smiles': 'A-UGLY-TYPO-C'},6,molecule,c053009a-9b1d-44a7-8267-f5befdb03eb0,2020-02-17 03:09:48.673496
43,create,"{'order': 0, 'fragment_id': '93d1dd17-8f7b-4f6...",7,molecule_fragment,cf8151dd-2e19-43a3-a175-4e3293f858d5,2020-02-17 03:09:48.748453
41,create,"{'order': 1, 'fragment_id': 'e23166bb-88c4-464...",8,molecule_fragment,25b3209f-4266-449b-9223-666fcba592e8,2020-02-17 03:09:48.748453
42,create,"{'order': 2, 'fragment_id': '10e88031-39e6-4a6...",9,molecule_fragment,a262f5e8-2286-41a6-81b6-371e08f1d4d3,2020-02-17 03:09:48.748453
40,update,"{'smiles': 'ABC', 'metadata': {}}",10,molecule,c053009a-9b1d-44a7-8267-f5befdb03eb0,2020-02-17 03:09:48.810674


We can see here from the event `rollback` everything is deleted and then the event are sequentially repeated until the we reach the given timestamp.